# Generate Markdown Documentation on RTX-KG2 with Kuzu

Use pip install found below for standalone use through external environments (such as Google Colab).

In [ ]:
# for installation in external runtime environments
!pip install "git+https://github.com/CU-DBMI/rtx-kg2-gateway"

In [1]:
import json
import os
import pathlib
import tarfile

import kuzu
from genson import SchemaBuilder

from notebooks import (
    download_file,
    extract_tar_gz,
    infer_generic_json_schema_from_object,
)

In [2]:
# set some variables for the work below
source_data_url = "https://github.com/CU-DBMI/rtx-kg2-gateway/releases/download/v0.0.1/kg2c_lite_2.8.4.full.with-metanames.dataset.kuzu.tar.gz"
target_dir = "../data"
target_database_path = f"{target_dir}/kg2c_lite_2.8.4.full.with-metanames.dataset.kuzu"

In [3]:
# create a target download path
pathlib.Path(target_dir).mkdir(exist_ok=True)

In [4]:
# niave check for existing database to avoid redownloading / extracting if possible
if not pathlib.Path(target_database_path).is_dir():
    downloaded_file = download_file(url=source_data_url, download_dir=target_dir)
    extract_dir = extract_tar_gz(
        tar_gz_path=f"{target_dir}/{downloaded_file}", output_dir=target_dir
    )

In [5]:
# init a Kuzu database and connection
db = kuzu.Database(target_database_path)
kz_conn = kuzu.Connection(db)

In [6]:
# gather table details from Kuzu database
df_table_names_by_type = kz_conn.execute(
    """
    /* SHOW_TABLES() is a special Kuzu function for
    sharing database table details. */
    CALL SHOW_TABLES()

    /* filter to REL_GROUP's and NODE table types */
    WHERE type IN ['REL_GROUP', 'NODE']

    RETURN name, type
    ORDER BY type, name ASC;
    """
).get_as_df()
df_table_names_by_type

,name,type
0,Activity,NODE
1,Agent,NODE
2,AnatomicalEntity,NODE
3,Behavior,NODE
4,BehavioralFeature,NODE
...,...,...
118,superclass_of,REL_GROUP
119,temporally_related_to,REL_GROUP
120,transcribed_from,REL_GROUP
121,translates_to,REL_GROUP


In [7]:
# gather node entity example from Kuzu database
example_node = (
    kz_conn.execute(
        """
        /* match on arbitrary node */
        MATCH (node)
        WHERE node.id = 'UMLS:C2459634'
        RETURN
            node.*;
        """
    )
    .get_as_df()
    .iloc[0]
    .to_dict()
)

example_node

{'node.id': 'UMLS:C2459634',
 'node.name': 'Oral Peripheral Mechanism Assessment using Other Equipment',
 'node.all_categories': ['biolink:Procedure'],
 'node.category': 'biolink:Procedure'}

In [8]:
""  # gather rel entity example from Kuzu database
example_rel = (
    kz_conn.execute(
        """
        /* match on arbitrary relationship */
        MATCH ()-[r:treats]-()
        WHERE r.id = 19799062
        RETURN
            r.*;
        """
    )
    .get_as_df()
    .iloc[0]
    .to_dict()
)

# form a dictionary showing the schema and values together
example_rel

{'r.qualified_object_aspect': '',
 'r.predicate': 'biolink:treats',
 'r.domain_range_exclusion': 'True',
 'r.qualified_object_direction': '',
 'r.id': 19799062,
 'r.primary_knowledge_source': 'infores:semmeddb',
 'r.qualified_predicate': ''}

In [9]:
def generate_entity_example_html_table(entity_json: dict) -> str:
    """
    Generates an HTML table for showing
    an example of entity data and schema.
    """
    return f"""<table>
<tr>
<th>Example data</th>
<th>Example data JSON schema</th>
</tr>
<td>

```json
{json.dumps(entity_json, indent=2)}
```

</td>
<td>

```json
{infer_generic_json_schema_from_object(entity_json)}
```

</td>
</tr>
</table>
"""

In [10]:
with open(
    "rtx-kg2-gateway-kuzu-database-details.md", mode="w", encoding="utf-8"
) as markdown_file:

    markdown_file.write(
        """# RTX-KG2-gateway Kuzu Database Schema Details

Please see below for details on the Kuzu database associated with RTX-KG2 created as part of this project.

## Table types

Tables below are provided as either NODE or REL_GROUP tables.
[REL_GROUPS](https://kuzudb.com/docusaurus/cypher/data-definition/create-table#create-rel-table-group) are specialized tables within Kuzu databases that are collections of NODE type pairs.
REL_GROUPS are shown by name below which can be referenced as part of Cypher queries (instead of the more verbose and many NODE type pairs).

"""
    )

    markdown_file.write(
        "\n\n".join(
            [
                f"""### {table_type} Tables

#### Example {table_type} Data and JSON Schema

{generate_entity_example_html_table(example_node) if table_type == "NODE" else generate_entity_example_html_table(example_rel)}

#### {table_type} Names

{table_md_content}
"""
                for table_type, table_md_content in {
                    table_type: df_table_names_by_type[
                        df_table_names_by_type["type"] == table_type
                    ][["name", "type"]].to_markdown(index=False)
                    for table_type in df_table_names_by_type["type"].unique().tolist()
                }.items()
            ]
        )
    )

In [11]:
# run an example query
kz_conn.execute(
    """
    MATCH (d:Disease)
    WHERE d.name = "Down syndrome"
    RETURN d.id, d.name, d.all_categories;
    """
).get_as_df()

,d.id,d.name,d.all_categories
0,MONDO:0008608,Down syndrome,[biolink:Disease]
